In [1]:
# Import dependencies
import os
from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-04-22 05:41:31.597327: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 05:41:31.913497: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745293292.013619    3928 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745293292.042747    3928 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745293292.282077    3928 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 19 intra-op, 12 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1745293295.793634    3928 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset, calculate_class_weights
from src.model.densnet.tensorflow_dense_net_basic import build_densenet

# Input Data
initial_epoch   = None
resume_training = False
checkpoint_path = None
set_name        = "SimplifiedDenseNetV1"
model_name      = f"{set_name}_3"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

class_weights = calculate_class_weights(train_ds, num_classes)

I0000 00:00:1745293295.832153    3928 cuda_executor.cc:479] failed to allocate 18.00PiB (20266198323167232 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745293295.832178    3928 cuda_executor.cc:479] failed to allocate 16.20PiB (18239577202360320 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745293295.832181    3928 cuda_executor.cc:479] failed to allocate 14.58PiB (16415619482124288 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745293295.832183    3928 cuda_executor.cc:479] failed to allocate 13.12PiB (14774057211789312 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745293295.832185    3928 cuda_executor.cc:479] failed to allocate 11.81PiB (13296650738991104 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1745293295.832186    3928 cuda_executor.cc:

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

2025-04-22 05:43:14.016985: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
2025-04-22 05:43:17.530927: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Counting samples: 71842 samples [00:36, 1958.14 samples/s]
Counting samples: 15431 samples [00:13, 1147.12 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryCrossentropy(from_logits=False)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

In [5]:
# Model Training
model          = build_densenet(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-04-22 05:43:32 - INFO - 
=== Training Started ===

2025-04-22 05:43:32 - INFO - Batch Size: 128
2025-04-22 05:43:32 - INFO - Optimizer: Adam
2025-04-22 05:43:32 - INFO - 

2025-04-22 05:43:32 - INFO - 
=== Starting Epoch 1 ===



Epoch 1/100


I0000 00:00:1745293416.843944    3992 service.cc:152] XLA service 0x78c5f8003260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745293416.843962    3992 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1745293416.861726    3992 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745293416.941815    3992 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-22 05:43:52.876964: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] ShuffleDatasetV3:13: Filling up shuffle buffer (this may take a while): 18929 of 25000
2025-04-22 05:43:56.135590: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:482] Shuffle buffer filled.
2025-04-22 05:43:58.847653: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to 

306/561 ━━━━━━━━━━━━━━━━━━━━ 1:04 253ms/step - accuracy: 0.8977 - auc: 0.5240 - f1_score: 0.0351 - loss: 0.7849 - precision: 0.1127 - recall: 0.0236

2025-04-22 05:46:22.929374: W tensorflow/core/lib/png/png_io.cc:92] PNG warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG


...

561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step - accuracy: 0.9143 - auc: 0.7912 - f1_score: 0.1331 - loss: 0.2010 - precision: 0.4968 - recall: 0.0945  

2025-04-22 07:50:01.936928: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 77078016 bytes after encountering the first element of size 77078016 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
2025-04-22 07:50:02 - INFO - 
=== Epoch 43 Summary ===
2025-04-22 07:50:02 - INFO - Time: 174.59s
2025-04-22 07:50:02 - INFO - Training   - accuracy: 0.9140 - auc: 0.7919 - f1_score: 0.1292 - loss: 0.2004 - precision: 0.4886 - recall: 0.0925 - learning_rate: 0.0000
2025-04-22 07:50:02 - INFO - Validation - accuracy: 0.9127 - auc: 0.7476 - f1_score: 0.1091 - loss: 0.2870 - precision: 0.4301 - recall: 0.0769
2025-04-22 07:50:02 - INFO - ==================================================



561/561 ━━━━━━━━━━━━━━━━━━━━ 175s 311ms/step - accuracy: 0.9143 - auc: 0.7912 - f1_score: 0.1331 - loss: 0.2010 - precision: 0.4968 - recall: 0.0945 - val_accuracy: 0.9127 - val_auc: 0.7476 - val_f1_score: 0.1091 - val_loss: 0.2870 - val_precision: 0.4301 - val_recall: 0.0769 - learning_rate: 1.0000e-06


2025-04-22 07:50:02 - INFO - 
=== Training Completed! ===

2025-04-22 07:50:02 - INFO - Final Metrics: accuracy: 0.9140 - auc: 0.7919 - f1_score: 0.1292 - loss: 0.2004 - precision: 0.4886 - recall: 0.0925 - val_accuracy: 0.9127 - val_auc: 0.7476 - val_f1_score: 0.1091 - val_loss: 0.2870 - val_precision: 0.4301 - val_recall: 0.0769



Epoch 43: early stopping
Restoring model weights from the end of the best epoch: 28.
